## マルチホライズン売買プラン：月次＋週次シグナル設計メモ

---

## 1. 取引コンセプト（再確認）
| 項目 | 内容 |
|------|------|
| 売買頻度 | **月次で銘柄選別 → 週次でタイミング** |
| 月次フェーズ | 月末にシグナル生成 → 翌月第1営業日に銘柄候補を確定 |
| 週次フェーズ | 毎週末に売買タイミングを判定 → 月内で 1〜2 回取引 |
| 対象銘柄 | 上場銘柄 20 社（流動性と業種分散を確保） |
| モデル出力 | 月次：翌1か月リターンスコア<br>週次：翌1週間の上げ下げ確率 |
| 運用スタイル | 半自動（モデルがスコアを出し、上位 N 銘柄を買い／下位を空売り or 見送り） |
| リスク管理 | 等金額（equal weight）＋最大ドローダウン枠でロット調整 |

---

## 2. シグナル vs. 特徴量
- **シグナル**：売買判断そのもの（例：12か月モメンタム）  
- **特徴量**：モデル学習用入力（シグナルをそのまま／派生を追加）  
- 月次と週次で **シグナル集合を分けて** 管理し、後段で統合スコアを作る。

---

## 3-A. 月次モデルに適したシグナル（例）

| カテゴリ | シグナル | 定義（終値を月足にリサンプル） | 目的 |
|----------|----------|---------------------------------|------|
| **モメンタム** | 12–1 モメンタム | 過去12ヶ月リターン − 過去1ヶ月リターン | 長期トレンド捉え |
| **バリュー** | PER / PBR 逆数 | 低バリュー銘柄優先 | 割安度 |
| **クオリティ** | ROE, 営業利益率 | 財務健全度 | 安定収益 |
| **ボラティリティ** | β（月次） | 市場感応度 | リスク分散 |

---

## 3-B. 週次モデルに適したシグナル（例）

> 月次フィルタを通過した銘柄にのみ計算

| カテゴリ | シグナル | 定義（週足ベース） | 目的 |
|----------|----------|-------------------|------|
| モメンタム | 4-1 週モメンタム | 短期トレンド捉え |
| オシレーター | RSI(14w) | オーバー/アンダー判定 |
| ボラ | ATR(5w) | エントリー幅設定 |

（※ 週次候補は先述の表から優先度が高いものを 5〜7 本選択）

---

## 4. シグナル計算・更新サイクル

| フェーズ | タイミング | 手順 |
|----------|-----------|------|
| **月次** | 月末引け後 | ① 日足→月足へリサンプル<br>② 月次シグナル計算 & スコア付け<br>③ 上位20銘柄を「監視リスト」に保存 |
| **週次** | 監視リスト銘柄のみ、毎週金曜引け後 | ① 日足→週足へリサンプル<br>② 週次シグナル計算<br>③ XGBで買い／売り判定 |
| **発注** | 翌週月曜寄付き | スコア上位 N を買い（下位を空売り or 見送り） |
| **モニタリング** | 週中 | OCO決済（利確＋損切）・損益管理 |

---

## 5. 必要カラムとデータ粒度

- **最低カラム**：`Date, Open, High, Low, Close, Adj Close, Volume`  
- **追加推奨**：`Dividends, Stock Splits, Fundamentals (PER, PBR, ROE)`  
- **粒度**：日足取得 → 月足と週足にリサンプリング（分足不要）

---

## 6. 次のアクション

1. **月足 & 週足リサンプル関数** を整備し、20銘柄×必要列を取得  
2. **月次シグナル計算モジュール** で候補リスト生成  
3. **週次シグナル計算 + 既存モデル** でタイミング判定  
4. Hit Rate／Sharpe／IR をモニタリングし、シグナルの取捨選択を継続

---

本ドキュメントは月次＋週次の二層構造に合わせて再編済み。  
既に対応している週次パートはそのまま流用し、月次フィルタ層を先に追加するだけで移行が可能。  